In [87]:
import pandas as pd
import numpy as np
import random, time
from statistics import mean

# Stats Data:

In [502]:
base_stats = pd.read_csv('base_stats.csv').fillna(0)
char_growth = pd.read_csv('char_growth.csv').fillna(0)
class_base = pd.read_csv('class_base.csv').fillna(0)
class_growth = pd.read_csv('class_growth.csv').fillna(0)
base_stats.head(3)

,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Alear,1,Dragon Child,0,22,4,6,0,5,7,5,3,5
1,Vander,1,Paladin,11,40,8,11,5,10,8,10,8,6
2,Clanne,1,Mage,0,19,4,1,8,11,9,4,7,4


In [601]:
sg_cols = char_growth.columns.tolist()
sg_cols

['Name', 'HP', 'Bld', 'Str', 'Mag', 'Dex', 'Spd', 'Def', 'Res', 'Lck']

In [606]:
class_growth.head()

,Name,Unique,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Dragon Child,Alear,10.0,5.0,10.0,0.0,10.0,15.0,10.0,10.0,5.0
1,Divine Dragon,Alear,10.0,5.0,10.0,0.0,10.0,15.0,15.0,15.0,10.0
2,Fell Child,"Alear, Veyle",10.0,0.0,10.0,25.0,10.0,5.0,10.0,25.0,0.0
3,Noble,Alfred,15.0,0.0,15.0,0.0,5.0,5.0,20.0,5.0,15.0
4,Avenir,Alfred,15.0,5.0,15.0,0.0,5.0,10.0,25.0,5.0,20.0


In [607]:
class_growth[sg_cols[1:]] = class_growth[sg_cols[1:]]*2
class_growth.head()

,Name,Unique,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Dragon Child,Alear,20.0,10.0,20.0,0.0,20.0,30.0,20.0,20.0,10.0
1,Divine Dragon,Alear,20.0,10.0,20.0,0.0,20.0,30.0,30.0,30.0,20.0
2,Fell Child,"Alear, Veyle",20.0,0.0,20.0,50.0,20.0,10.0,20.0,50.0,0.0
3,Noble,Alfred,30.0,0.0,30.0,0.0,10.0,10.0,40.0,10.0,30.0
4,Avenir,Alfred,30.0,10.0,30.0,0.0,10.0,20.0,50.0,10.0,40.0


# Levelling:

In [609]:
def level_up(base_stats, char_growth, class_base, class_growth, char, adv_tgt, 
             base_tgt=10, promo_class=None, verbose=False):
    # Get stat growth column names:
    sg_cols = char_growth.columns.tolist()
    # Include modifier for HP:
    hp_mod = max(40/adv_tgt, 1)
    # Alter class growth rates if character is Jean (due to his ability):
    if char == 'Jean':
        class_growth[sg_cols[1:]] = class_growth[sg_cols[1:]]*2
    
    # Extract character base stats:
    char_base = base_stats[base_stats.Name==char].reset_index(drop=True)
    # Add in row to track character growth:
    char_base = pd.concat([char_base, pd.DataFrame({'Name':'Growth'}, index=[0])]).reset_index(drop=True).fillna(0)
    # Character-specific stat growth:
    char_g = char_growth[char_growth.Name==char][sg_cols[1:]].values
    # Counters to keep track of cycles through  classes:
    grwth_counter = 0
    adv_counter = 0
    # Return the current character vars:
    class_type = class_base[class_base['Name']==char_base['Class'][0]]['Type'].values[0]
    char_class = char_base['Class'].values[0]   
    # Track starting character level:
    if verbose:
        print("Character starts at level: ", char_base['Level'].values[0])
        
    # If character is a base class and >= level 10, promote immediately:
    if char_base['Level'][0] >= base_tgt and class_type == 'Base':
        if verbose:
            print("Promoting %s to %s immediately." % (char, promo_class))
        adv_tgt -= ((char_base['Level'][0] - base_tgt) + 1)
        char_base['Level'] = 1
        char_base['Class'] = promo_class  
        class_diffs = change_class(class_base, sg_cols, char_class, promo_class, char)
        char_base[sg_cols[1:]] += class_diffs
          
    elif class_type == 'Advanced' and char_class != promo_class:
        if verbose:
            print("Using second seal to %s immediately." % (promo_class))
        char_base['Level'] = 1
        char_base['Class'] = promo_class  
        class_diffs = change_class(class_base, sg_cols, char_class, promo_class, char)
        char_base[sg_cols[1:]] += class_diffs
        adv_tgt -= 1
    elif class_type == 'Advanced' and char_class == promo_class:
        adv_tgt -= 1
    elif class_type == 'Special':
        # Add base_tgt to make up for lack of base training:
        adv_tgt += base_tgt
        # Less the characters current level:
        adv_tgt -= char_base['Level'].values[0]
    
    # If the character has already trained internal levels, deduce this from total target:
    int_chg = max(char_base['Internal'][0] - base_tgt, 0)
    if verbose:
        print("Change due to internal level: ", int_chg)
    adv_tgt -= int_chg
    
    

    # For each level, increase stats according to growth:
    while adv_tgt > 0:
        # Current character variables:
        char_class = char_base['Class'].values[0]
        class_type = class_base[class_base['Name']==char_class]['Type'].values[0]
        
        # Calculate stat growth according to class and character stat growth:
        class_g = class_growth[class_growth['Name']==char_class][sg_cols[1:]].values
        stat_g = pd.DataFrame(np.clip(np.add(char_g, class_g), 0, 100), columns = sg_cols[1:])
        
        # For each stat, grow it according to specific stat growth:
        for col in range(1, len(sg_cols)):
            growth = stat_g[sg_cols[col]].values[0]
            rand_n = random.randint(0, 100)
            # If roll is lower than growth number, then grow stat:
            if rand_n <= growth:
                char_base[sg_cols[col]] += 1
                
        # Increase level stat:
        char_base['Level'] += 1  
        grwth_counter += 1
        if class_type == 'Advanced' or class_type == 'Special':
            adv_tgt -= 1
        
        # Promote char if level 10 and still in a base class:
        if char_base['Level'][0] >= base_tgt and class_type == 'Base':
            # Reset character level to 1:
            char_base['Level'] = 1
            if verbose:
                print("%s became a level 10 %s, resetting to Level 1..." % (char, char_base['Class'].values[0]))
            # Change character class:
            char_base['Class'] = promo_class
            class_diffs = change_class(class_base, sg_cols, char_class, promo_class, char)
            char_base[sg_cols[1:]] += class_diffs
            adv_tgt -= 1
            if verbose:
                print("%s is now a %s" %(char_base['Name'].values[0], promo_class))
            
        # Use second seal to reset lev at 20 for advanced class:
        elif char_base['Level'][0] == 20 and class_type == 'Advanced':
            char_base['Level'] = 1
            adv_counter += 1
            adv_tgt -= 1
            if verbose:
                print("Resetting %s's Advanced class at level 20..." % char)
                print("%s is now a %s +%s" %(char, promo_class, adv_counter))
                        
        elif char_base['Level'][0] == 40 and class_type == 'Special':
            char_base['Level'] = 1
            adv_counter += 1
            adv_tgt -= 1
            if verbose:
                print("Resetting Special %s class at level 40..." % char)
                print("%s is now a %s +%s" %(char, promo_class, adv_counter))
    
    # Change level growth:
    char_base.loc[1, 'Level'] = grwth_counter
    
    # Calculate total rating:
    total_rating = int(char_base[sg_cols[2:]].iloc[0].sum()) + int(char_base['HP'].iloc[0]/hp_mod)
    if verbose:
        print("%s has a total rating of: %s" % (char, total_rating))
    
    return total_rating, char_base
                

In [150]:
def change_class(class_base, sg_cols, curr_class, new_class, char):
    cb_curr = class_base[class_base['Name']==curr_class]
    
    # Extracting unique class info:
    if len(cb_curr) > 1:
        cb_curr = cb_curr[cb_curr['Unique']==char]
    
    cb_new = class_base[class_base['Name']==new_class]
    
    # Calculate class base stat differences:
    diffs = cb_new[sg_cols[1:]].iloc[0] - cb_curr[sg_cols[1:]].iloc[0]
    
    return diffs

# Experiments:

In [513]:
# Static Vars:
char = 'Celine'
promo = 'Vidame'
levels = 20

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Celine became a level 10 Noble, resetting to Level 1...
Celine is now a Vidame
Resetting Celine's Advanced class at level 20...
Celine is now a Vidame +1
Celine has a total rating of: 169


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Celine,1,Vidame,0.0,36.0,5.0,21.0,15.0,19.0,23.0,16.0,22.0,30.0
1,Growth,24,Vidame,0.0,14.0,1.0,14.0,6.0,10.0,13.0,11.0,15.0,19.0


In [508]:
# Static Vars:
char = 'Celine'
promo = 'Griffin Knight'
levels = 20

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Celine became a level 10 Noble, resetting to Level 1...
Celine is now a Griffin Knight
Resetting Celine's Advanced class at level 20...
Celine is now a Griffin Knight +1
Celine has a total rating of: 177


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Celine,1,Griffin Knight,0.0,42.0,6.0,15.0,15.0,26.0,29.0,14.0,23.0,28.0
1,Growth,24,Griffin Knight,0.0,20.0,2.0,8.0,6.0,17.0,19.0,9.0,16.0,17.0


In [527]:
# Static Vars:
char = 'Diamant'
promo = 'Hero'
levels = 20

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Character starts at level:  11.0
Promoting Diamant to Hero immediately.
Level change:  2.0
Diamant has a total rating of: 151


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Diamant,19,Hero,0.0,50.0,11.0,22.0,4.0,19.0,24.0,16.0,13.0,17.0
1,Growth,18,Hero,0.0,18.0,2.0,9.0,1.0,6.0,11.0,6.0,7.0,7.0


In [529]:
# Static Vars:
char = 'Yunaka'
promo = 'Thief'
adv_tgt = 20

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, adv_tgt, promo_class=promo, verbose=True)
df

Character starts at level:  6.0
Yunaka has a total rating of: 156


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Yunaka,30.0,Thief,0.0,38.0,9.0,19.0,13.0,18.0,30.0,12.0,20.0,16.0
1,Growth,24.0,0,0.0,13.0,4.0,11.0,8.0,4.0,18.0,3.0,13.0,8.0


In [535]:
# Static Vars:
char = 'Kagetsu'
promo = 'Swordmaster'
levels = 20

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Character starts at level:  1.0
Change due to internal level:  5.0
Kagetsu has a total rating of: 188


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Kagetsu,15.0,Swordmaster,15.0,45.0,12.0,22.0,8.0,32.0,32.0,20.0,15.0,25.0
1,Growth,14.0,0,0.0,10.0,3.0,5.0,3.0,10.0,10.0,6.0,5.0,8.0


In [536]:
# Static Vars:
char = 'Saphir'
promo = 'Griffin Knight'
levels = 20

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Character starts at level:  7.0
Using second seal to Griffin Knight immediately.
Change due to internal level:  9.0
Saphir has a total rating of: 188


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Saphir,11,Griffin Knight,19.0,55.0,15.0,24.0,5.0,30.0,29.0,21.0,16.0,21.0
1,Growth,10,Griffin Knight,0.0,4.0,0.0,0.0,3.0,7.0,10.0,2.0,7.0,6.0


In [442]:
# Static Vars:
char = 'Kagetsu'
promo = 'Swordmaster'
levels = 10

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Kagetsu has a total rating of: 164


,Name,Level,Class,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Kagetsu,10.0,Swordmaster,42.0,10.0,22.0,7.0,30.0,29.0,17.0,17.0,22.0
1,Growth,9.0,0,7.0,1.0,5.0,2.0,8.0,7.0,3.0,7.0,5.0


# Iterations:

In [343]:
# Advanced classes:
uniq_adv = class_base[(class_base['Unique'] !=0) & (class_base['Type'] == 'Advanced')][['Name', 'Unique']]
ua_dict = dict(zip(uniq_adv.Unique, uniq_adv.Name))
non_uniq = class_base[(class_base['Unique'] ==0) & (class_base['Type'] == 'Advanced')]['Name'].tolist()

In [338]:
non_uniq[0:3]

['Swordmaster', 'Hero', 'Halberdier']

In [460]:
char_list = base_stats.Name.tolist()
char_list.remove('Veyle')
print(len(char_list))
char_list[0:3]

35


['Alear', 'Vander', 'Clanne']

In [538]:
iters = 100
adv_tgt = 20
all_ratings = []

for char in char_list:
    char_ratings = []
    
    for pro_class in non_uniq:
    
        class_rating = []
        for i in range(iters):
            r, _ = level_up(base_stats, char_growth, class_base, class_growth, char, adv_tgt, promo_class=pro_class)
            class_rating.append(r)
        print("%s | %s | Rt. %s | %s lvls." % (char, pro_class, mean(class_rating), adv_tgt+10))
        char_ratings.append(mean(class_rating))
    
    uc_rating = []
    try:
        unique_class = ua_dict[char]
        for i in range(iters):
            r, _ = level_up(base_stats, char_growth, class_base, class_growth, char, adv_tgt, promo_class=unique_class)
            uc_rating.append(r)
        print("%s | %s | Rt. %s | %s lvls." % (char, unique_class, mean(class_rating), adv_tgt+10))
    except KeyError:
        uc_rating.append(0)
    char_ratings.append(mean(uc_rating))
    
    all_ratings.append(char_ratings)   
    
# Create dataframe of results:
res_df = pd.DataFrame([item[0:len(non_uniq)] for item in all_ratings], columns= non_uniq)
res_df.insert(0, 'Name', char_list)
res_df['Unique'] = [item[-1] for item in all_ratings]
res_df.head()

Alear | Swordmaster | Rt. 159.85 | 30 lvls.
Alear | Hero | Rt. 157.45 | 30 lvls.
Alear | Halberdier | Rt. 159.77 | 30 lvls.
Alear | Royal Knight | Rt. 169.65 | 30 lvls.
Alear | Berserker | Rt. 160.24 | 30 lvls.
Alear | Warrior | Rt. 162.53 | 30 lvls.
Alear | Sniper | Rt. 154.84 | 30 lvls.
Alear | Bow Knight | Rt. 158.18 | 30 lvls.
Alear | General | Rt. 168.88 | 30 lvls.
Alear | Great Knight | Rt. 170.17 | 30 lvls.
Alear | Paladin | Rt. 165.7 | 30 lvls.
Alear | Wolf Knight | Rt. 164.85 | 30 lvls.
Alear | Sage | Rt. 160.08 | 30 lvls.
Alear | Mage Knight | Rt. 164.63 | 30 lvls.
Alear | Martial Master | Rt. 161.11 | 30 lvls.
Alear | High Priest | Rt. 164.67 | 30 lvls.
Alear | Griffin Knight | Rt. 173.58 | 30 lvls.
Alear | Wyvern Knight | Rt. 164.13 | 30 lvls.
Alear | Divine Dragon | Rt. 164.13 | 30 lvls.
Vander | Swordmaster | Rt. 130.66 | 30 lvls.
Vander | Hero | Rt. 129.41 | 30 lvls.
Vander | Halberdier | Rt. 131.55 | 30 lvls.
Vander | Royal Knight | Rt. 139.01 | 30 lvls.
Vander | Berser

Yunaka | Bow Knight | Rt. 159.13 | 30 lvls.
Yunaka | General | Rt. 158.25 | 30 lvls.
Yunaka | Great Knight | Rt. 158.74 | 30 lvls.
Yunaka | Paladin | Rt. 160.04 | 30 lvls.
Yunaka | Wolf Knight | Rt. 159 | 30 lvls.
Yunaka | Sage | Rt. 159.23 | 30 lvls.
Yunaka | Mage Knight | Rt. 159.17 | 30 lvls.
Yunaka | Martial Master | Rt. 158.59 | 30 lvls.
Yunaka | High Priest | Rt. 158.78 | 30 lvls.
Yunaka | Griffin Knight | Rt. 159.34 | 30 lvls.
Yunaka | Wyvern Knight | Rt. 159.5 | 30 lvls.
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Lev

Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change

Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change

Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Level change:  1.0
Alcryst | Sage | Rt. 148.96 | 30 lvls.
Level change:  1.0
Level change:  1.0
Level chang

...

Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0


Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0


Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0
Level change:  10.0


Lindon | Sage | Rt. 172.63 | 30 lvls.
Lindon | Mage Knight | Rt. 174.88 | 30 lvls.
Lindon | Martial Master | Rt. 171.25 | 30 lvls.
Lindon | High Priest | Rt. 175.32 | 30 lvls.
Lindon | Griffin Knight | Rt. 183.86 | 30 lvls.
Lindon | Wyvern Knight | Rt. 175.94 | 30 lvls.
Saphir | Swordmaster | Rt. 176.28 | 30 lvls.
Saphir | Hero | Rt. 173.95 | 30 lvls.
Saphir | Halberdier | Rt. 177.51 | 30 lvls.
Saphir | Royal Knight | Rt. 183.08 | 30 lvls.
Saphir | Berserker | Rt. 175.23 | 30 lvls.
Saphir | Warrior | Rt. 178.6 | 30 lvls.
Saphir | Sniper | Rt. 171.14 | 30 lvls.
Saphir | Bow Knight | Rt. 174.64 | 30 lvls.
Saphir | General | Rt. 184.48 | 30 lvls.
Saphir | Great Knight | Rt. 185 | 30 lvls.
Saphir | Paladin | Rt. 182.1 | 30 lvls.
Saphir | Wolf Knight | Rt. 180.14 | 30 lvls.
Saphir | Sage | Rt. 177.84 | 30 lvls.
Saphir | Mage Knight | Rt. 180.57 | 30 lvls.
Saphir | Martial Master | Rt. 175.86 | 30 lvls.
Saphir | High Priest | Rt. 179.74 | 30 lvls.
Saphir | Griffin Knight | Rt. 187.55 | 30 lv

,Name,Swordmaster,Hero,Halberdier,Royal Knight,Berserker,Warrior,Sniper,Bow Knight,General,Great Knight,Paladin,Wolf Knight,Sage,Mage Knight,Martial Master,High Priest,Griffin Knight,Wyvern Knight,Unique
0,Alear,159.85,157.45,159.77,169.65,160.24,162.53,154.84,158.18,168.88,170.17,165.70,164.85,160.08,164.63,161.11,164.67,173.58,164.13,160.84
1,Vander,130.66,129.41,131.55,139.01,131.65,133.27,126.07,129.49,140.44,141.04,137.79,137.16,131.65,134.24,131.38,134.95,143.92,136.89,0.00
2,Clanne,155.91,152.90,156.66,165.97,157.95,160.40,152.10,155.71,166.71,166.16,164.46,162.02,158.22,160.24,156.58,162.77,166.84,161.90,0.00
3,Framme,156.86,152.08,155.79,163.59,155.53,159.27,149.18,153.57,164.65,165.62,161.47,160.19,155.60,158.97,154.38,160.04,167.97,159.81,0.00
4,Alfred,152.55,149.35,152.92,159.92,153.20,154.63,147.45,151.66,162.64,162.18,159.64,156.56,152.66,156.17,152.41,156.71,164.16,158.45,166.53


In [587]:
res_df.sort_values(by="Hero", ascending=False)

,Name,Swordmaster,Hero,Halberdier,Royal Knight,Berserker,Warrior,Sniper,Bow Knight,General,Great Knight,Paladin,Wolf Knight,Sage,Mage Knight,Martial Master,High Priest,Griffin Knight,Wyvern Knight,Unique
32,Mauvier,261.09,258.30,262.01,265.94,261.29,264.02,256.34,260.63,269.49,270.63,267.62,265.74,261.66,265.02,261.43,266.86,274.29,266.97,0.00
18,Kagetsu,185.37,183.66,186.54,193.84,186.51,188.18,180.20,184.33,193.95,194.55,192.31,190.29,186.85,188.56,184.28,189.79,197.48,190.58,0.00
25,Merrin,177.45,175.33,178.10,185.12,177.27,180.00,171.58,175.65,186.05,186.14,183.85,183.03,178.07,180.42,177.45,181.91,189.24,182.56,0.00
31,Saphir,176.28,173.95,177.51,183.08,175.23,178.60,171.14,174.64,184.48,185.00,182.10,180.14,177.84,180.57,175.86,179.74,187.55,181.31,0.00
21,Pandreo,174.62,171.70,175.13,181.63,174.17,177.15,169.04,172.36,181.76,182.28,180.01,178.07,174.56,177.57,173.95,177.90,184.75,179.37,0.00
30,Lindon,172.02,170.47,172.46,178.99,172.14,175.74,166.96,170.01,180.81,181.47,178.54,175.37,172.63,174.88,171.25,175.32,183.86,175.94,0.00
29,Goldmary,169.84,167.47,170.28,176.76,169.72,172.57,164.51,167.55,178.23,178.22,175.46,173.91,170.16,172.70,169.45,172.65,181.61,174.41,0.00
24,Panette,167.32,165.36,167.95,175.16,166.88,170.45,162.17,165.43,175.58,175.82,173.30,171.78,167.18,170.26,166.31,169.36,178.07,171.09,0.00
8,Chloe,165.39,163.67,167.23,174.32,166.29,168.47,160.76,163.54,175.53,174.60,172.76,171.18,166.05,170.03,166.47,170.57,180.18,171.47,0.00
28,Rosado,165.96,162.73,164.97,172.31,166.06,167.87,161.01,164.04,173.60,174.57,171.26,168.72,167.19,168.86,164.78,169.06,177.09,170.45,0.00


In [590]:
res_df.to_csv('data/20_adv_levels.csv')

In [583]:
# Static Vars:
char = 'Clanne'
promo = 'Sniper'
levels = 40

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Character starts at level:  1.0
Change due to internal level:  0
Clanne became a level 10 Mage, resetting to Level 1...
Clanne is now a Sniper
Resetting Clanne's Advanced class at level 20...
Clanne is now a Sniper +1
Resetting Clanne's Advanced class at level 20...
Clanne is now a Sniper +2
Clanne has a total rating of: 245


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Clanne,1,Sniper,0.0,53.0,7.0,32.0,9.0,46.0,35.0,22.0,20.0,21.0
1,Growth,47,Sniper,0.0,34.0,3.0,31.0,1.0,35.0,26.0,18.0,13.0,17.0


In [584]:
# Static Vars:
char = 'Alcryst'
promo = 'Sniper'
levels = 40

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Character starts at level:  10.0
Promoting Alcryst to Sniper immediately.
Change due to internal level:  0
Resetting Alcryst's Advanced class at level 20...
Alcryst is now a Sniper +1
Resetting Alcryst's Advanced class at level 20...
Alcryst is now a Sniper +2
Alcryst has a total rating of: 224


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Alcryst,1,Sniper,0.0,59.0,10.0,27.0,6.0,45.0,29.0,23.0,9.0,16.0
1,Growth,38,Sniper,0.0,31.0,4.0,16.0,4.0,28.0,17.0,15.0,4.0,8.0


In [585]:
# Static Vars:
char = 'Lapis'
promo = 'Sniper'
levels = 40

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Character starts at level:  10.0
Promoting Lapis to Sniper immediately.
Change due to internal level:  0
Resetting Lapis's Advanced class at level 20...
Lapis is now a Sniper +1
Resetting Lapis's Advanced class at level 20...
Lapis is now a Sniper +2
Lapis has a total rating of: 252


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Lapis,1,Sniper,0.0,59.0,6.0,32.0,17.0,43.0,34.0,22.0,17.0,22.0
1,Growth,38,Sniper,0.0,33.0,1.0,21.0,15.0,31.0,20.0,15.0,11.0,15.0


## 40 Levels:

In [586]:
iters = 1000
adv_tgt = 40
all_ratings40 = []

for char in char_list:
    char_ratings = []
    
    for pro_class in non_uniq:
    
        class_rating = []
        for i in range(iters):
            r, _ = level_up(base_stats, char_growth, class_base, class_growth, char, adv_tgt, promo_class=pro_class)
            class_rating.append(r)
        print("%s | %s | Rt. %s | %s lvls." % (char, pro_class, mean(class_rating), adv_tgt+10))
        char_ratings.append(mean(class_rating))
    
    uc_rating = []
    try:
        unique_class = ua_dict[char]
        for i in range(iters):
            r, _ = level_up(base_stats, char_growth, class_base, class_growth, char, adv_tgt, promo_class=unique_class)
            uc_rating.append(r)
        print("%s | %s | Rt. %s | %s lvls." % (char, unique_class, mean(class_rating), adv_tgt+10))
    except KeyError:
        uc_rating.append(0)
    char_ratings.append(mean(uc_rating))
    
    all_ratings40.append(char_ratings)   
    
# Create dataframe of results:
res_40 = pd.DataFrame([item[0:len(non_uniq)] for item in all_ratings40], columns= non_uniq)
res_40.insert(0, 'Name', char_list)
res_40['Unique'] = [item[-1] for item in all_ratings40]
res_40.head()

Alear | Swordmaster | Rt. 255.146 | 50 lvls.
Alear | Hero | Rt. 253.79 | 50 lvls.
Alear | Halberdier | Rt. 257.954 | 50 lvls.
Alear | Royal Knight | Rt. 267.216 | 50 lvls.
Alear | Berserker | Rt. 262.295 | 50 lvls.
Alear | Warrior | Rt. 264.196 | 50 lvls.
Alear | Sniper | Rt. 252.954 | 50 lvls.
Alear | Bow Knight | Rt. 255.845 | 50 lvls.
Alear | General | Rt. 271.862 | 50 lvls.
Alear | Great Knight | Rt. 271.075 | 50 lvls.
Alear | Paladin | Rt. 266.247 | 50 lvls.
Alear | Wolf Knight | Rt. 263.496 | 50 lvls.
Alear | Sage | Rt. 253.944 | 50 lvls.
Alear | Mage Knight | Rt. 258.035 | 50 lvls.
Alear | Martial Master | Rt. 255.318 | 50 lvls.
Alear | High Priest | Rt. 259.633 | 50 lvls.
Alear | Griffin Knight | Rt. 268.46 | 50 lvls.
Alear | Wyvern Knight | Rt. 264.693 | 50 lvls.
Alear | Divine Dragon | Rt. 264.693 | 50 lvls.
Vander | Swordmaster | Rt. 215.717 | 50 lvls.
Vander | Hero | Rt. 214.209 | 50 lvls.
Vander | Halberdier | Rt. 218.2 | 50 lvls.
Vander | Royal Knight | Rt. 226.671 | 50 l

Yunaka | Royal Knight | Rt. 245.584 | 50 lvls.
Yunaka | Berserker | Rt. 245.975 | 50 lvls.
Yunaka | Warrior | Rt. 245.87 | 50 lvls.
Yunaka | Sniper | Rt. 245.865 | 50 lvls.
Yunaka | Bow Knight | Rt. 246.007 | 50 lvls.
Yunaka | General | Rt. 245.611 | 50 lvls.
Yunaka | Great Knight | Rt. 246.08 | 50 lvls.
Yunaka | Paladin | Rt. 245.983 | 50 lvls.
Yunaka | Wolf Knight | Rt. 245.419 | 50 lvls.
Yunaka | Sage | Rt. 245.398 | 50 lvls.
Yunaka | Mage Knight | Rt. 245.684 | 50 lvls.
Yunaka | Martial Master | Rt. 245.464 | 50 lvls.
Yunaka | High Priest | Rt. 246.181 | 50 lvls.
Yunaka | Griffin Knight | Rt. 245.699 | 50 lvls.
Yunaka | Wyvern Knight | Rt. 245.488 | 50 lvls.
Alcryst | Swordmaster | Rt. 237.333 | 50 lvls.
Alcryst | Hero | Rt. 235.466 | 50 lvls.
Alcryst | Halberdier | Rt. 239.003 | 50 lvls.
Alcryst | Royal Knight | Rt. 247.868 | 50 lvls.
Alcryst | Berserker | Rt. 244.148 | 50 lvls.
Alcryst | Warrior | Rt. 245.117 | 50 lvls.
Alcryst | Sniper | Rt. 233.839 | 50 lvls.
Alcryst | Bow Knig

Fogado | Warrior | Rt. 259.764 | 50 lvls.
Fogado | Sniper | Rt. 248.413 | 50 lvls.
Fogado | Bow Knight | Rt. 251.728 | 50 lvls.
Fogado | General | Rt. 267.193 | 50 lvls.
Fogado | Great Knight | Rt. 266.146 | 50 lvls.
Fogado | Paladin | Rt. 262.233 | 50 lvls.
Fogado | Wolf Knight | Rt. 259.439 | 50 lvls.
Fogado | Sage | Rt. 250.483 | 50 lvls.
Fogado | Mage Knight | Rt. 254.62 | 50 lvls.
Fogado | Martial Master | Rt. 252.524 | 50 lvls.
Fogado | High Priest | Rt. 255.346 | 50 lvls.
Fogado | Griffin Knight | Rt. 263.796 | 50 lvls.
Fogado | Wyvern Knight | Rt. 260.542 | 50 lvls.
Fogado | Cupido | Rt. 260.542 | 50 lvls.
Pandreo | Swordmaster | Rt. 270.952 | 50 lvls.
Pandreo | Hero | Rt. 269.263 | 50 lvls.
Pandreo | Halberdier | Rt. 272.616 | 50 lvls.
Pandreo | Royal Knight | Rt. 281.168 | 50 lvls.
Pandreo | Berserker | Rt. 277.136 | 50 lvls.
Pandreo | Warrior | Rt. 277.934 | 50 lvls.
Pandreo | Sniper | Rt. 267.287 | 50 lvls.
Pandreo | Bow Knight | Rt. 270.121 | 50 lvls.
Pandreo | General | R

Lindon | Royal Knight | Rt. 273.877 | 50 lvls.
Lindon | Berserker | Rt. 270.566 | 50 lvls.
Lindon | Warrior | Rt. 272.108 | 50 lvls.
Lindon | Sniper | Rt. 260.511 | 50 lvls.
Lindon | Bow Knight | Rt. 263.752 | 50 lvls.
Lindon | General | Rt. 279.098 | 50 lvls.
Lindon | Great Knight | Rt. 278.156 | 50 lvls.
Lindon | Paladin | Rt. 274.063 | 50 lvls.
Lindon | Wolf Knight | Rt. 270.836 | 50 lvls.
Lindon | Sage | Rt. 262.95 | 50 lvls.
Lindon | Mage Knight | Rt. 266.845 | 50 lvls.
Lindon | Martial Master | Rt. 263.984 | 50 lvls.
Lindon | High Priest | Rt. 267.781 | 50 lvls.
Lindon | Griffin Knight | Rt. 276.414 | 50 lvls.
Lindon | Wyvern Knight | Rt. 272.27 | 50 lvls.
Saphir | Swordmaster | Rt. 265.474 | 50 lvls.
Saphir | Hero | Rt. 264.338 | 50 lvls.
Saphir | Halberdier | Rt. 267.74 | 50 lvls.
Saphir | Royal Knight | Rt. 275.471 | 50 lvls.
Saphir | Berserker | Rt. 268.979 | 50 lvls.
Saphir | Warrior | Rt. 271.608 | 50 lvls.
Saphir | Sniper | Rt. 261.657 | 50 lvls.
Saphir | Bow Knight | Rt. 

,Name,Swordmaster,Hero,Halberdier,Royal Knight,Berserker,Warrior,Sniper,Bow Knight,General,Great Knight,Paladin,Wolf Knight,Sage,Mage Knight,Martial Master,High Priest,Griffin Knight,Wyvern Knight,Unique
0,Alear,255.146,253.790,257.954,267.216,262.295,264.196,252.954,255.845,271.862,271.075,266.247,263.496,253.944,258.035,255.318,259.633,268.460,264.693,258.830
1,Vander,215.717,214.209,218.200,226.671,222.757,223.825,212.775,215.758,231.491,230.246,226.632,223.572,214.502,218.213,215.818,219.814,228.289,224.556,0.000
2,Clanne,240.301,237.896,242.172,251.236,247.096,247.987,237.355,240.370,256.052,254.615,251.311,248.148,238.464,241.745,240.092,243.888,253.108,249.318,0.000
3,Framme,245.569,242.923,247.274,256.351,251.571,253.258,241.937,245.093,260.893,260.283,255.649,252.983,243.708,247.261,245.348,249.050,258.088,254.191,0.000
4,Alfred,247.694,245.589,249.023,258.204,254.338,255.180,243.931,247.615,263.322,262.313,258.034,255.150,245.795,249.596,247.288,251.770,260.361,256.061,265.234


In [588]:
res_40.sort_values(by="Hero", ascending=False)

,Name,Swordmaster,Hero,Halberdier,Royal Knight,Berserker,Warrior,Sniper,Bow Knight,General,Great Knight,Paladin,Wolf Knight,Sage,Mage Knight,Martial Master,High Priest,Griffin Knight,Wyvern Knight,Unique
32,Mauvier,373.785,372.267,376.410,380.316,381.222,382.186,370.862,374.690,389.749,388.779,385.010,382.307,372.663,376.458,374.364,378.588,387.715,383.755,0.000
18,Kagetsu,286.143,284.712,288.240,296.630,293.247,294.320,282.885,285.867,301.416,300.225,296.748,293.630,284.448,288.608,286.115,290.057,298.499,295.167,0.000
25,Merrin,271.453,269.714,273.604,281.820,277.747,278.756,268.070,271.520,286.618,286.040,281.645,278.752,270.162,274.094,271.619,275.046,284.032,280.067,0.000
21,Pandreo,270.952,269.263,272.616,281.168,277.136,277.934,267.287,270.121,286.270,285.091,281.373,277.567,268.416,273.114,270.593,274.598,283.560,279.282,0.000
28,Rosado,266.982,265.917,269.911,277.818,272.356,274.908,263.635,266.494,282.188,281.410,277.779,274.500,265.881,270.086,267.253,271.346,280.123,276.186,0.000
8,Chloe,266.057,264.504,267.813,276.924,270.821,274.141,263.045,265.862,281.862,280.732,276.253,273.784,264.211,268.305,266.625,269.761,278.910,274.737,0.000
31,Saphir,265.474,264.338,267.740,275.471,268.979,271.608,261.657,264.899,278.802,279.221,275.253,272.476,263.908,268.052,265.925,269.006,277.592,273.720,0.000
30,Lindon,264.192,262.948,265.819,273.877,270.566,272.108,260.511,263.752,279.098,278.156,274.063,270.836,262.950,266.845,263.984,267.781,276.414,272.270,0.000
33,Anna,261.824,260.178,264.238,272.085,268.011,269.934,258.475,261.638,277.504,276.617,272.693,269.392,259.743,263.796,262.006,265.952,274.200,270.724,0.000
24,Panette,261.766,260.027,263.597,271.923,266.568,269.239,258.345,261.276,277.170,275.683,271.417,268.958,259.780,263.842,261.577,264.924,274.223,270.409,0.000


In [589]:
res_40.to_csv('data/40_adv_levels.csv')

In [591]:
# Static Vars:
char = 'Anna'
promo = 'Sage'
levels = 40

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Character starts at level:  5.0
Change due to internal level:  0
Anna became a level 10 Axe Fighter, resetting to Level 1...
Anna is now a Sage
Resetting Anna's Advanced class at level 20...
Anna is now a Sage +1
Resetting Anna's Advanced class at level 20...
Anna is now a Sage +2
Anna has a total rating of: 266


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Anna,1,Sage,0.0,47.0,8.0,8.0,44.0,36.0,34.0,14.0,48.0,27.0
1,Growth,43,Sage,0.0,18.0,1.0,-2.0,42.0,27.0,27.0,10.0,43.0,24.0


In [593]:
# Static Vars:
char = 'Citrinne'
promo = 'Sage'
levels = 40

r, df = level_up(base_stats, char_growth, class_base, class_growth, char, levels, promo_class=promo, verbose=True)
df

Character starts at level:  10.0
Promoting Citrinne to Sage immediately.
Change due to internal level:  0
Resetting Citrinne's Advanced class at level 20...
Citrinne is now a Sage +1
Resetting Citrinne's Advanced class at level 20...
Citrinne is now a Sage +2
Citrinne has a total rating of: 227


,Name,Level,Class,Internal,HP,Bld,Str,Mag,Dex,Spd,Def,Res,Lck
0,Citrinne,1,Sage,0.0,49.0,8.0,3.0,44.0,25.0,22.0,13.0,40.0,23.0
1,Growth,38,Sage,0.0,25.0,4.0,1.0,29.0,12.0,12.0,10.0,28.0,13.0


In [610]:
iters = 1000
adv_tgt = 40
all_ratings_j = []

for char in ['Jean']:
    char_ratings = []
    
    for pro_class in non_uniq:
    
        class_rating = []
        for i in range(iters):
            r, _ = level_up(base_stats, char_growth, class_base, class_growth, char, adv_tgt, promo_class=pro_class)
            class_rating.append(r)
        print("%s | %s | Rt. %s | %s lvls." % (char, pro_class, mean(class_rating), adv_tgt+10))
        char_ratings.append(mean(class_rating))
    
    uc_rating = []
    try:
        unique_class = ua_dict[char]
        for i in range(iters):
            r, _ = level_up(base_stats, char_growth, class_base, class_growth, char, adv_tgt, promo_class=unique_class)
            uc_rating.append(r)
        print("%s | %s | Rt. %s | %s lvls." % (char, unique_class, mean(class_rating), adv_tgt+10))
    except KeyError:
        uc_rating.append(0)
    char_ratings.append(mean(uc_rating))
    
    all_ratings_j.append(char_ratings)

jean_df = pd.DataFrame([item[0:len(non_uniq)] for item in all_ratings_j], columns= non_uniq)
jean_df.insert(0, 'Name', char_list)
jean_df

Jean | Swordmaster | Rt. 378.757 | 50 lvls.
Jean | Hero | Rt. 379.24 | 50 lvls.
Jean | Halberdier | Rt. 439.177 | 50 lvls.
Jean | Royal Knight | Rt. 444.241 | 50 lvls.
Jean | Berserker | Rt. 389.564 | 50 lvls.
Jean | Warrior | Rt. 420.54 | 50 lvls.
Jean | Sniper | Rt. 372.121 | 50 lvls.
Jean | Bow Knight | Rt. 377.008 | 50 lvls.
Jean | General | Rt. 373.808 | 50 lvls.
Jean | Great Knight | Rt. 431.092 | 50 lvls.
Jean | Paladin | Rt. 414.246 | 50 lvls.
Jean | Wolf Knight | Rt. 411.173 | 50 lvls.
Jean | Sage | Rt. 337.48 | 50 lvls.
Jean | Mage Knight | Rt. 416.741 | 50 lvls.
Jean | Martial Master | Rt. 390.128 | 50 lvls.
Jean | High Priest | Rt. 337.805 | 50 lvls.
Jean | Griffin Knight | Rt. 401.391 | 50 lvls.
Jean | Wyvern Knight | Rt. 449.696 | 50 lvls.


ValueError: Length of values (35) does not match length of index (1)

In [612]:
jean_df = pd.DataFrame([item[0:len(non_uniq)] for item in all_ratings_j], columns= non_uniq)
jean_df.insert(0, 'Name', ['Jean'])
jean_df

,Name,Swordmaster,Hero,Halberdier,Royal Knight,Berserker,Warrior,Sniper,Bow Knight,General,Great Knight,Paladin,Wolf Knight,Sage,Mage Knight,Martial Master,High Priest,Griffin Knight,Wyvern Knight
0,Jean,378.757,379.24,439.177,444.241,389.564,420.54,372.121,377.008,373.808,431.092,414.246,411.173,337.48,416.741,390.128,337.805,401.391,449.696
